In [1]:
from ultralytics import YOLO

In [3]:
import numpy as np
dummy = np.zeros((640, 640, 3))
model.predict(dummy)

Loading Models/yolodet_int8/best.engine for TensorRT inference...
[04/09/2025-15:51:28] [TRT] [I] Loaded engine size: 7 MiB
[04/09/2025-15:51:28] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +169, now: CPU 0, GPU 172 (MiB)

0: 640x640 (no detections), 1.3ms
Speed: 34.3ms preprocess, 1.3ms inference, 35.4ms postprocess per image at shape (1, 3, 640, 640)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'potato'}
 obb: None
 orig_img: array([[[          0,           0,           0],
         [          0,           0,           0],
         [          0,           0,           0],
         ...,
         [          0,           0,           0],
         [          0,           0,           0],
         [          0,           0,           0]],
 
        [[          0,           0,           0],
         [          0,           0,           0],
         [          0,           0,           0],
         ...,
         [          0,           0,           0],
         [          0,           0,           0],
         [          0,           0,           0]],
 
        [[          0,           0,           0],
         [          0,           0,           0],
         [          0,           0,           0],
         ...,
         [         

In [8]:
import ffmpegcv
import torch.nn.functional as F
cap = ffmpegcv.toCUDA(ffmpegcv.VideoCaptureNV('video.mp4', pix_fmt='nv12', gpu=0), tensor_format='chw', gpu=0)
while True:
    ret, frame_CHW_CUDA = cap.read_torch()
    if not ret:
        break
    frame_CHW_CUDA = frame_CHW_CUDA.unsqueeze(0)
    new_height = 640  
    new_width = 640  
    resized_tensor = F.interpolate(frame_CHW_CUDA, size=(new_height, new_width), mode='bilinear', align_corners=False)
    results = model.track(resized_tensor, imgsz=640)
cap.release()

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 255.0. Dividing input by 255.
[04/09/2025-15:57:46] [TRT] [E] IExecutionContext::executeV2: Error Code 1: Cuda Runtime (invalid resource handle)
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 255.0. Dividing input by 255.
[04/09/2025-15:57:46] [TRT] [E] IExecutionContext::executeV2: Error Code 1: Cuda Runtime (invalid resource handle)
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 255.0. Dividing input by 255.
[04/09/2025-15:57:46] [TRT] [E] IExecutionContext::executeV2: Error Code 1: Cuda Runtime (invalid resource handle)
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 255.0. Dividing input by 255.
[04/09/2025-15:57:46] [TRT] [E] IExecutionContext::executeV2: Error Code 1: Cuda Runtime (invalid resource handle)
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 255.0. Dividing input by 255.
[04/0

In [7]:
import ffmpegcv
import time
from pynvml import *
from contextlib import contextmanager
import torch.nn.functional as F

@contextmanager
def gpu_monitoring():
    """Context manager for GPU monitoring"""
    class GPUMonitor:
        def __init__(self, handle):
            self.handle = handle
            
        def get_stats(self):
            try:
                util = nvmlDeviceGetUtilizationRates(self.handle)
                mem = nvmlDeviceGetMemoryInfo(self.handle)
                return {
                    'gpu_util': util.gpu,
                    'mem_util': util.memory,
                    'mem_used': mem.used / 1024**2  # MB
                }
            except NVMLError as e:
                print(f"GPU Stats Error: {e}")
                return {'gpu_util': -1, 'mem_util': -1, 'mem_used': -1}
    
    handle = None
    try:
        nvmlInit()
        handle = nvmlDeviceGetHandleByIndex(0)
        yield GPUMonitor(handle)
    except NVMLError as e:
        print(f"NVML Init Error: {e}")
        yield None
    finally:
        if handle is not None:
            try:
                nvmlShutdown()
            except NVMLError as e:
                print(f"NVML Shutdown Error: {e}")

# cap1 = ffmpegcv.toCUDA(ffmpegcv.VideoCaptureNV('video.mp4', pix_fmt='nv12'))
# cap2 = ffmpegcv.toCUDA(ffmpegcv.VideoCaptureNV('video.mp4', pix_fmt='nv12'))

start_time = time.time()
frame_count = 0
cap = ffmpegcv.toCUDA(ffmpegcv.VideoCaptureNV('video.mp4', pix_fmt='nv12', gpu=0), tensor_format='chw', gpu=0)

print(f"Processing on GPU 0 | {cap.width}x{cap.height} @ {cap.fps:.1f} FPS")
print("Frame | GPU% | MEM% | Used(MB) | FPS")
print("------------------------------------")
try:
    with gpu_monitoring() as monitor, cap:
        while True:
            ret, frame_CHW_CUDA = cap.read_torch()
            if not ret:
                break
            frame_CHW_CUDA = frame_CHW_CUDA.unsqueeze(0)
            new_height = 640  
            new_width = 640  
            resized_tensor = F.interpolate(frame_CHW_CUDA, size=(new_height, new_width), mode='bilinear', align_corners=False)/255.0
            results = model.track(resized_tensor, imgsz=640, verbose=False)
            if frame_count % 30 == 0:
                stats = monitor.get_stats() if monitor else {'gpu_util': -1, 'mem_util': -1, 'mem_used': -1}
                elapsed = max(0.001, time.time() - start_time)
                fps = frame_count / elapsed
                
                print(
                    f"{frame_count:5d} | "
                    f"{stats['gpu_util']:3d}% | "
                    f"{stats['mem_util']:3d}% | "
                    f"{stats['mem_used']:7.1f} | "
                    f"{fps:5.1f}"
                )  
            frame_count+=1   
                
except KeyboardInterrupt:
    print("\nProcess interrupted")
finally:
    total_time = time.time() - start_time
    print("\nSummary:")
    print(f"Frames processed: {frame_count}")
    print(f"Total time: {total_time:.1f}s")
    print(f"Average FPS: {frame_count/total_time:.1f}")

Processing on GPU 0 | 3840x2160 @ 30.0 FPS
Frame | GPU% | MEM% | Used(MB) | FPS
------------------------------------
[04/09/2025-15:53:40] [TRT] [E] IExecutionContext::executeV2: Error Code 1: Cuda Runtime (invalid resource handle)
    0 |   0% |   0% |  1853.8 |   0.0
[04/09/2025-15:53:40] [TRT] [E] IExecutionContext::executeV2: Error Code 1: Cuda Runtime (invalid resource handle)
[04/09/2025-15:53:40] [TRT] [E] IExecutionContext::executeV2: Error Code 1: Cuda Runtime (invalid resource handle)
[04/09/2025-15:53:40] [TRT] [E] IExecutionContext::executeV2: Error Code 1: Cuda Runtime (invalid resource handle)
[04/09/2025-15:53:41] [TRT] [E] IExecutionContext::executeV2: Error Code 1: Cuda Runtime (invalid resource handle)
[04/09/2025-15:53:41] [TRT] [E] IExecutionContext::executeV2: Error Code 1: Cuda Runtime (invalid resource handle)
[04/09/2025-15:53:41] [TRT] [E] IExecutionContext::executeV2: Error Code 1: Cuda Runtime (invalid resource handle)
[04/09/2025-15:53:41] [TRT] [E] IExecuti